In [1]:
from sentence_transformers import SentenceTransformer
import pandas as pd
import numpy as np
import faiss
import torch
device = torch.device("cuda:0") if torch.cuda.is_available() else torch.device("cpu")

c:\Users\parth\anaconda3\envs\wwrecommender\lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


In [2]:
def config():
    model = SentenceTransformer('nomic-ai/nomic-embed-text-v1', trust_remote_code=True).to(device)

C:\Users\parth\.cache\huggingface\modules\transformers_modules\nomic-ai\nomic-bert-2048\e55a7d4324f65581af5f483e830b80f34680e8ff\modeling_hf_nomic_bert.py:95: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related 

In [5]:
def get_postings():
    jobs = pd.read_csv('WaterlooWorks-Desirability-Predictor/all_jobs.csv').dropna()[['ID', 'Job Title', 'Organization', 'Openings', 'City', 'Level', 'Applications', 'Job Responsibilities', 'Required Skills', 'Cover Letter Needed']]
    postings = (
        'ID: ' + jobs['ID'].astype(str) + 
        ', Job Title:  ' + jobs['Job Title'] + 
        ', Organization: ' + jobs['Organization'] + 
        ', Openings: ' + jobs['Openings'].astype(str) + 
        ', City: ' + jobs['City'] +
        ', Level: ' + jobs['Level'] +
        ', Applications: ' + jobs['Applications'].astype(str) +
        ', Job Responsibilities: ' + jobs['Job Responsibilities'] +
        ', Required Skills: ' + jobs['Required Skills'] + 
        ', Cover Letter Needed: ' + jobs['Cover Letter Needed'].astype(str)
    ).tolist()
    return postings

KeyboardInterrupt: 

In [ ]:
def embed_postings():
    embeddings = model.encode(postings, device=device)
    dims = embeddings.shape[1]
    index = faiss.IndexFlatL2(dims)
    index.add(embeddings)

    embeddings = model.encode(postings, device=device)
dims = embeddings.shape[1]
index = faiss.IndexFlatL2(dims)
index.add(embeddings)

In [ ]:
def query():
    search_query = input('What jobs do you want to see: ')
    k = int(input('How many jobs do you want to see: '))
    svec = np.array(model.encode(search_query)).reshape(1,-1)
    _, pos = index.search(svec,k=k)
    return jobs.iloc(pos[0])